This notebook shows how to design a transmon pocket (single pad design) in qiskit metal. The notebook only need **TransmonPocket_Single** function from a relative path '../../resources'.

In [1]:
%reload_ext autoreload
%autoreload 2

In [32]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('Customized_Components')
from rounded_single_pad import Round_TransmonPocket_Single as transmon
import Transmon_property as trans_p
import Transmon_specifications as jj

import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [3]:
import os
import shutil
import time

In [4]:
import numpy as np
import astropy.units as u
import astropy.constants as c

import pandas as pd
import matplotlib.pyplot as plt

# %matplotlib inline

In [5]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
# from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.analyses.quantization import EPRanalysis

# from qiskit_metal.toolbox_metal.parsing import *

In [9]:
# from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
# from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround


In [10]:
# pd.read_csv("Frequencies from metal pad size.csv",).plot(x= 'Pad_len (mm)', y = 'EPR Frequency (GHz)')
# plt.ylabel('Frequency (GHz)')
# plt.show()

The target frequency is around 4.2GHz. To get this frequency, we need the pad size to be around 0.275mm.

## Global variables

In [6]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.45

In [7]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '20 mm'
design.chips.main.size['size_y'] = '10 mm'

design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

design.variables['trace_width'] = '12.4 um'
cpw_pin_width = 12.4*u.um
design.variables['trace_gap'] = '7 um'
cpw_gap = 7*u.um

design.variables['pad_buffer_radius'] = '30 um'
design.variables['buffer_resolution'] = '10'
design.variables['connection_pad_buffer_radius'] = '2 um'
# design.chips['main']['material'] = 'Silicon'

gui = MetalGUI(design)

05:24PM 12s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Populating font family aliases took 145 ms. Replace uses of missing font family "Courier" with one that exists to avoid this cost. 



Component-specific variables

In [8]:
qb_options = dict(
pad_pocket_distance_top = '40um',
connection_pads = dict(
    a = dict(loc_W  = 0, 
            loc_H =  1, 
            pad_gap = '15um',
            pad_height = '15um',
            pad_width = '80um',
            pad_cpw_extent =  '10um',
            pocket_rise = '0um',
            cpw_extend = '0um',
            pocket_extent = '0um',)))

TQ_options = dict(prime_width = design.variables['cpw_width'],
               prime_gap = design.variables['cpw_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '60um',
               coupling_space = '5um',
               open_termination=False,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False)

CPW_options = Dict(trace_width = design.variables['trace_width'],
               trace_gap  = design.variables['trace_gap'],
        total_length='5 mm',
        hfss_wire_bonds = True,
        q3d_wire_bonds = True,
        fillet='30 um',
        lead = dict(start_straight='20um', end_straight = '50um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='TQ1', pin='second_end')), )

pin_inputs = Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='TQ1', pin='second_end'))

trans_options = Dict(hfss_wire_bonds = True,
                     q3d_wirebonds = True,
               pin_inputs=Dict(
                 start_pin=Dict(
                     component='TQ1',
                     pin='prime_start'),
                 end_pin=Dict(
                     component='TQ2',
                     pin='prime_end')))


## Design

Single pad transmon

In [9]:
design.delete_all_components()

In [10]:
design.overwrite_enabled = True

size = 0.25*u.mm
gap = 30*u.um
size = size.to(u.um)
pocket_width = size+2*gap


op = Dict(
    pos_x = '0',
    pad_height = '{}um'.format(size.value),
    pos_y = '0',
    pad_width = '{}'.format(size),
    pocket_width = '{}'.format(pocket_width), 
    **qb_options)

q1 = transmon(design,'Q1',options = op)

gui.rebuild()
gui.zoom_on_components(['Q1'])


In [11]:
op = Dict(
    pos_x = '1mm',
    pad_height = '{}um'.format(size.value),
    pos_y = '0',
    pad_width = '{}'.format(size),
    pocket_width = '{}'.format(pocket_width), 
    **qb_options)

q2 = transmon(design,'Q2',options = op)

gui.rebuild()
gui.zoom_on_components(['Q2'])

Coupled Line Tee

In [12]:
TQ_options['down_length'] = '40 um'
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='-0.1 mm',
                                             pos_y='0.5 mm',
                                             mirror = True,
                                             coupling_length='120 um', **TQ_options))
TQ1.options['open_termination'] = False

gui.rebuild()
gui.autoscale()

  return lib.buffer(

  return lib.buffer(



In [13]:
TQ2 = CoupledLineTee(design, 'TQ2', options=dict(pos_x='0.9 mm',
                                             pos_y='0.5 mm',
                                             mirror = True,
                                             coupling_length='120 um', **TQ_options))
TQ2.options['open_termination'] = False

gui.rebuild()
gui.autoscale()

  return lib.buffer(

  return lib.buffer(



Function that constructs CPW

In [23]:
def slice_data(data, freq):
    diff = freq-4
    ind = round(diff/0.2)
    return data.iloc[ind]

# def qb_cpw_construction(q, TQ, freq, guesses = guesses):
    

In [19]:
def construct_cpw_qubit(q, TQ, freq, displacement = '0um', guess_path = guess_path,sim = True):
    guess_all = pd.read_csv(guess_path)
    guesses = slice_data(guess_all, freq)
    size = guesses['Sizes (um)']*u.um
    buffer = guesses['Buffers (um)']*u.um
    offset = guesses['Offsets (mm)']
    coupling_len = guesses['Coupling_len(um)']*u.um
    coupling_gap = guesses['Coupling_gap(um)']*u.um
    Lj = guesses['Ljs']
    Cj = jj.find_junction_capacitance(int(Lj[:-2])*u.nH)
    
    Cj1 = str(Cj.to(u.fF).value)+' fF'
    size = size.to(u.um)

    TQx = guesses['TQx']
    TQy = guesses['TQy']
    TQ_mir = guesses['TQ_mir']
    small = guesses['Small']
    
    Construct_CPW(q, TQ, size, offset, coupling_len, coupling_gap, Lj, Cj1, TQx,TQy, small, TQ_mir,displacement, buffer, sim)
    

In [29]:
def Construct_CPW(q, TQ, pad_size, offset, extend, gapp, Lj, Cj, TQx,TQy, small, TQ_mir, displacement = '0um', buffer = 150*u.um, sim  = True):
    gap1 = 0.056
    gap = 30*u.um
    size = pad_size.to(u.um)
    pocket_width = size+2*gap
    cpw_name = 'cpw_'+ q.name[-1:]
    design.delete_component(cpw_name)
    coupling_len = extend
    q.options['pad_height'] = '{}'.format(size)
    q.options['pad_width'] = '{}'.format(size)
    q.options['pocket_width'] = '{}'.format(pocket_width)
    q.options['connection_pads']['a']['pad_width'] = '{}'.format(coupling_len)
    q.options['connection_pads']['a']['pad_height'] = '30um-{}'.format(gapp)
    q.options['connection_pads']['a']['pad_gap'] = '{}'.format(gapp)
    q.options.hfss_inductance = Lj
    q.options.q3d_inductance =  Lj
    q.options.hfss_capacitance = Cj
    q.options.q3d_capacitance =  Cj

    l_name = 'Lj'+ q.name[-1:]
    c_name = 'Cj'+ q.name[-1:]

    if sim:
        eig_all.sim.renderer.options[l_name] = Lj
        eig_all.sim.renderer.options[c_name] = Cj
        eig_all.sim.setup.vars = {l_name:Lj, c_name:Cj}
    
    gui.rebuild()

    TQ.options.pos_x = TQx + '+' + displacement
    q.options.pos_x = displacement
    q.options.pos_y = '-'+TQy
    TQ.options.mirror = TQ_mir
    TQ.options.pos_y = '0um'
    gui.rebuild()

    anchors = trans_p.anchor_CPW_round(q, buffer, gap1, 2, small = small, last_offset = offset)
    design.delete_component(cpw_name)
    gui.rebuild()
    
    pin_inputs = Dict(
                start_pin=Dict(component=q.name, pin='a'),
                end_pin=Dict(component=TQ.name, pin='second_end'))

    CPW_options['pin_inputs'] = pin_inputs

    qa = RouteMixed(design, 'cpw_'+q.name[-1:], options = Dict(anchors = anchors, **CPW_options))

    gui.rebuild()

    
    # length = trans_p.find_total_len(qa, q1, TQ1, count_extend=False)
    # return (trans_p.find_actual_frequency(length, cpw_pin_width, cpw_gap), length)

In [36]:
construct_cpw_qubit(q1, TQ1, 5, displacement = '0um', guess_path = guess_path, sim = False)

05:38PM 56s INFO [delete_component]: Called delete_component cpw_1, but such a component is not in the design cache dictionary of components.


In [33]:
construct_cpw_qubit(q2, TQ2, 4.4, displacement = '1mm', guess_path = guess_path, sim = False)

05:38PM 23s INFO [delete_component]: Called delete_component cpw_2, but such a component is not in the design cache dictionary of components.


# Simulation

Import Qiskit Packages

In [30]:
from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.analyses.quantization import EPRanalysis

Set the right path for Ansys Screenshots

In [29]:
original = r'C:\Users\slab\Desktop\Wendy-qiskit-code\ansys.png'
path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\Ansys_screenshoots\Simulation_0404'

In [ ]:
components  = ['Q1','cpw_1','TQ1','Q2','TQ2','cpw_2','cpw_mid']
nmode = 4
datas = pd.DataFrame()

In [16]:
components_all = ['Q1','Q2','cpw_1','cpw_2','TQ1','TQ2','CPW_left','CPW_right', 'CPW_mid','wb_left','wb_right']

In [38]:
all_freqs = np.arange(4,6,0.2)

In [39]:
from itertools import combinations
p = combinations(all_freqs,2)

In [ ]:
# path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\Ansys_screenshoots\Simulation_0314\loop'
gap1 = 0.056
datas = pd.DataFrame()
for f1,f2 in p:
    dat = {}
    dat['guess_path'] = guess_path

    construct_cpw_qubit(q1, TQ1, f1, displacement = '0um', guess_path = guess_path, sim = True)
    construct_cpw_qubit(q2, TQ2, f2, displacement = '1mm', guess_path = guess_path, sim = True)
    TQ1.options['open_termination'] = False
    gui.rebuild()

    c1.sim.run(components=components)#, open_terminations=[('Q1', 'a')])#, ('Q1', 'bus1'), ('Q1', 'bus2')])
    c1.sim.capacitance_matrix 
    
    c1.sim._get_results_from_renderer()
    c_mat = c1.sim.capacitance_matrix
    renderer_q3d.clean_active_design()
    c1  = c_mat['a_connector_pad_Q1']['pad_top_Q1']
    c2  = c_mat['a_connector_pad_Q1']['pad_top_Q2']
    c12  = c_mat['pad_top_Q1']['pad_top_Q2']

    dat['C_Q1_Q2(fF)'] = c12
    dat['C_Q1_g(fF)'] = c1
    dat['C_Q2_g(fF)'] = c2
    
    count = 0
    while count<4:
        try:
            eig_all.sim.run(name="all_freqs", components=components)#, open_terminations=[('Q1', 'a')])
        except:
            print('count')
            renderer_hfss.clean_active_design()
            count += 1
            if count>=3:
                dat['Freq'+str(i+1)+'(GHz)'] = 0
                for i in range(nmode):
                    dat['Freq_EPR'+str(i+1)+'(MHz)'] = 0
                    for j in range(i+1):
                        dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = 0
                break
        else:
            convergence = pd.read_csv('hfss_eig_f_convergence.csv')
            conv = convergence.dropna()
        
            ind = list(conv.keys())[1:]
            for i in range(nmode):
                freq = conv[ind[(i)]].values[-1]
                dat['Freq'+str(i+1)+'(GHz)'] = freq
                
            for i in range(nmode):
                    hfss.modeler._modeler.ShowWindow()
                    hfss.set_mode(i+1,'Setup')
                    hfss.plot_ansys_fields('main')
                    hfss.save_screenshot()
                    hfss.clear_fields(['main'])
                    
                    actual = r'C:\Users\slab\Desktop\Wendy-qiskit-code\f1{:.3f}_f2{:.3f}_1mm.png'.format(f1,f2)
                    os.rename(original, actual)
                    fail = True
                    i = 0
                    while fail:

                        try:
                            dest = shutil.move(actual, path)
                        except:
                            i+= 1
                            os.rename(actual,(actual+'{}'.format(i)))
                            actual = (actual+'{}'.format(i))
                        else:
                            fail = False
                    
            #Run the EPR analysis
            try:
                del eig_all.setup.junctions['jj']
            except:
                print('saddd')
            eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                            Lj_variable='Lj1', Cj_variable='Cj1')
            eig_all.setup.sweep_variable = 'Lj1'
            eig_all.run_epr()

            #Save the EPR data
            chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
            freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]
            
            renderer_hfss.clean_active_design()
            for i in range(nmode):
                freq = freq_EPR[i]
                dat['Freq_EPR'+str(i+1)+'(MHz)'] = freq
                for j in range(i+1):
                    dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = chi[i][j]

            data = pd.DataFrame(dat, index = [0])
            datas = pd.concat([datas,data], ignore_index=True)
            break


    datas.to_csv('data\two_qubit_1mm_0403.csv')


# A huge loop

In [34]:
xs = np.linspace(0.65,14.5,10)
components = ['Q1','Q2','cpw_1','cpw_2','TQ1','TQ2'] 
components_all = ['Q1','Q2','cpw_1','cpw_2','TQ1','TQ2',
                  'CPW_left','CPW_right', 'CPW_mid','wb_left','wb_right']
#we are not simulating the feedline here since it contains a mode close to the qubit frequency

In [17]:
c = LOManalysis(design, "q3d")
c.sim.setup.min_passes  = 2
c.sim.setup.percent_error = 10

In [18]:
eig_all = EPRanalysis(design, 'hfss')
eig_all.sim.setup.min_passes = 5
eig_all.sim.setup.max_delta_f = 0.2

In [19]:
hfss = eig_all.sim.renderer
original = r'C:\Users\slab\Desktop\Wendy-qiskit-code\ansys.png'
path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\Ansys_screenshoots\Simulation_0304'

debugging

In [29]:
# eig_all.clear
# renderer_hfss.clean_active_design()

## Save Design to GDS

In [18]:
a_gds = design.renderers.gds

In [19]:
a_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '100um',
  'view_in_file': {'main': {1: True}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [20]:
a_gds.options['path_filename'] = 'Customized_Components\Fake_Junctions.GDS'

In [21]:

a_gds.options['short_segments_to_not_fillet'] = 'False'
a_gds.export_to_gds('One pad design.gds')

1